In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from functools import partial


# Computación paralela

En la clase pasada vimos como ganar rendimiento en operaciones SIMD usando NumPy

También aprendimos a conectar con lenguaje de bajo nivel usando Cython

Existe otra forma en que podemos ganar rendimiento en problemas que son "limitados en CPU"

El requisito adicional es que estos problemas sean separables

> Un problema es **separable** si puede dividirse en **subproblemas** que pueden resolverse de forma **independiente**

Al ser independientes significa que podemos resolverlos **al mismo tiempo**, es decir resolver cada uno sin esperar el resultado de los demás

> Arquitectura de computadores: Hoy en día incluso los CPU de sobremesa son en realidad **múltiples CPU** unidos

Es decir que

> Podemos escribir programas que aprovechan los CPU multi-nucleo para resolver problemas separables en un menor tiempo

Esto es lo que llamamos [**computación paralela**](https://computing.llnl.gov/tutorials/parallel_comp/#Whatis)

> En la práctica muchos problemas de computación científica (modelamiento, simulación) son paralelizables o incluso "masivamente paralelizables"

### Paralelizando una rutina

Asumiendo que el problema al que nos enfrentamos es limitado en CPU el primer paso es 

> Hacer *profiling* para encontrar los cuellos de botella

Luego de esto debemos

> Estudiar las zonas críticas y detectar oportunidades para paralelizar

El objetivo es encontrar sectores del programa que sean separables

Algunas preguntas típicas que pueden servir para esto son:
- ¿Existen ciclos `for` donde las iteraciones son independientes entre si?
- ¿Es posible descomponer la operación o los datos?
- ¿Existe una estructura de tipo pipeline?

Si alguna de estas respuestas es afirmativa entonces lo que resta es usar alguna herramienta de programación paralela para reescribir dicho sector del programa

A continuación veremos algunas herramientas para Python

# Python y rendimiento | Parte 3

En esta clase veremos distintas formas para paralelizar código escrito en Python

Antes de empezar debemos estar al tanto de que

> El manejo de memoria de CPython no es *thread-safe*

Es por esto que todo código escrito en Python está sujeto a un **mutex** que lo proteje: **Global Interpreter Lock (GIL)**

> El [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) obliga a ejecutar solo un hilo de código Python a la vez

Además 

> El código escrito en Python no tiene control sobre el GIL

Por esta razón no es directo ni fácil que un proceso Python puedo usar múltiples CPU

En esta clase exploraremos dos alternativas generales

> **1:** No manipular el GIL y usar **multiples procesos**: *ipyparallel* y *joblib*


> **2:** Levantar el GIL con Cython y usar **múltiples hilos**: *openmp*, *joblib*


Existe una tercera alternativa más accesible pero exclusiva para hacer **algebra lineal en paralelo** con NumPy

> **3:** Compilar NumPy contra una librería de algebra lineal de alto rendimiento (MKL, ATLAS, Openblas)

Estas librerías usan código de bajo nivel que levanta el GIL

### Diferencias entre computación multi-proceso y multi-hilo

- Multi-proceso: Levantar **varios procesos** de Python (fork)
    - Los procesos tiene su propio espacio de memoria y su propio GIL
- Multi-hilo: Levantar varios hilos en **un proceso** de Python
    - Los hilos comparten memoria
    - Los hilos no se pueden ejecutar en paralelo a menos de que levanten el GIL (Cython)


# Computación paralela con IPython: *ipyparallel*

[ipyparallel](https://ipyparallel.readthedocs.io/en/latest/) es un paquete independiente pero complementario de IPython para hacer computación multi-proceso

#### Instalación

Si tienes conda 

    conda install ipyparallel
    
Si usas pip

    pip install ipyparallel

Esto debería instalar en tu sistema unos ejecutables llamados `ipcluster`, `ipcontroller` e `ipengine`

Adicionalmente, en las versiones más nuevas, se crea una interfaz en la pestaña "Ipython clusters" del servidor jupyter llamada

Si la interfaz no aparece, se puede forzar con
    
    ipcluster nbextension enable
    
    
#### Conceptos y uso básico

*ipyparallel* considera varios elementos, los más importantes son:
- Engine: Es el encargado de correr código. Es una extensión del kernel de IPython
- Controller: Es una interfaz para comunicarnos con el/los engine/s. La conexión se hace a través del objeto `Client`

Para iniciar un controlador de forma automática abrimos un terminal y escribimos

    ipcluster start -n 4
    
o usamos los controles que se encuentran en la pestaña "IPython clusters" del servidor jupyter

Con esto hemos creado un controlador y cuatro engines, todos en nuestra máquina (localhost)

#### Creación de un cliente

In [ ]:
# Importamos ipyparallel
import ipyparallel as ipp
# Creamos la clase cliente
rc = ipp.Client()
# Verificamos que se hayan iniciado nuestro engines
display(rc.ids)

Cada engine tiene una id asociada

Para enviarle trabajo a los engines debemos crear una intefaz llamada [`View`](https://ipyparallel.readthedocs.io/en/latest/details.html#views)

Existen dos tipos de `View`: [*Direct*](https://ipyparallel.readthedocs.io/en/latest/direct.html#) y [*Task*](https://ipyparallel.readthedocs.io/en/latest/task.html#)

- La primera es controlada de forma explicita por el usuario
- La segunda es controlada por el sistema para *balancear la carga*
    
#### Enviando  trabajos usando interfaz Directa

Una `View` de tipo *Direct* requiere que el usuario especifique los engines que va a usar

Esto se hace de forma similar a los *slices* en listas/ndarray


Para crear una interfaz que utilice
- todas las engines, usamos `rc[:]`    
- las dos primeras engines, usamos `rc[:2]`

La vista puede hacerse bloqueante o no bloqueante (asíncrona) modificando el atributo booleano `block`

Una vista "bloqueante" espera a que el resultado de todos los engines sean retornado para devolver el control

In [ ]:
# Creamos una view con
dview = rc[:]
# Por defecto es asíncrono (no bloqueante)
display(dview.block)
# Lo podemos cambiar con
dview.block = True
display(dview.block)

Los trabajos se envían usando las funciones de la `View` directa

- `apply`, `apply_sync`, `apply_async`: Ejecutan una función con argumentos
- `map`, `map_sync`, `map_async`: Ejecutan una función sobre una secuencia
    
Los apellidos `sync`  y `async` cambian el flag del view momentaneamente

- Cuando trabajamos en forma síncrona el resultado retorna al final de la ejecuación
- Cuando trabajamos de forma asíncrona se retorna un objeto [`AsyncResult`](https://ipyparallel.readthedocs.io/en/latest/asyncresult.html#parallel-asyncresult) que puede ser consultado más tarde por el resultado

#### Funciones de Python en paralelo con `apply`

Se ocupa como 

    rc[:].apply(f, *args, **kwargs)

    
Por ejemplo usando una función anónima

In [ ]:
dview.apply(lambda x, y: x+" "+y, x="Hola", y="Mundo")

#### Compartiendo módulos y datos con los engines

Es importante tener claro que

> Los procesos en los engines no comparten memoria y no ven las variables de nuestro entorno local

Por ejemplo si queremos usar una función del módulo `os`

In [ ]:
#import os # Este import no lo ven los engines

def funcion():
    import os # Este si
    return os.getpid() 

# Cada uno tiene un pid distinto
dview.apply(funcion)

Podemos precargar un módulo en todos los engines con la función `sync_imports()` 

Los módulos se mantienen en el entorno de los engines

In [ ]:
with dview.sync_imports(local=True): 
    import os
# El módulo quedará importado también en nuestro ambiente local

# Ahora ya no necesitamos importar os
def funcion2(): 
    return os.getpid() 

dview.apply(funcion2)

Podemos limpiar las variables y módulos de los engines con `clear`

In [ ]:
# Limpiamos el entorno de las engines
dview.clear()
# Ahora esto ya no funciona
dview.apply(funcion2)

#### Pasando un dato a todos los engines

Para enviar un objeto de Python que hayamos definido en el ambiente local podemos usar la función `push`

El objeto tiene que ser un diccionario

Luego podemos usar `pull` si queremos extraer una variable remota

Estas funciones tienen el atributo `targets` que permite apuntar a un subconjunto de *engines*

In [ ]:
a = 100 # Esto no existe dentro de los engines

dview.push({'a': a}) # Ahora está en todos los engines

def funcion3(): 
    return a**2

display(dview.apply(funcion3)) # Ahora la función retorna correctamente

display(dview.pull('a', targets=[0,1]))

#### Distribuyendo datos a los engines

Si queremos distribuir datos en los engines podemos usar `scatter`

Luego podemos recuperar su valor usando `gather`

In [ ]:
# Una lista con 7 elementos que será distribuida en los 4 engines usando scatter
dview.scatter('c', np.array(range(10)))


def funcion3(): 
    global y # Creo una variable en el workspace del engine
    y = c**2 # Le doy un valor
    return y

display(dview.apply(funcion3))

# Recuperamos la salida con gather
display(dview.gather('y'))

Es posible distribuir arreglos de NumPy

> Los arreglos de NumPy no se copian, se traspasan *read-only*

In [ ]:
datos = np.random.randn(100, 100)
dview.scatter('data', datos)

def funcion4(): 
    # data[0, 0] = 0 # No podemos hacer esto!
    return data.shape

# Se particiona en 4 matrices por fila (row-major)
display(dview.apply(funcion4))

Si queremos hacer modificaciones inplace tenemos que hacer una copia local

In [ ]:
datos = np.random.randn(1000, 1000)
dview.scatter('data', datos)

def funcion5(): 
    global data
    if not data.flags.writeable:
        data = data.copy()
    data[0, 0] = 0 # No podemos hacer esto!
    return data

# Se particiona en 4 matrices por fila (row-major)
datos = np.concatenate(dview.apply(funcion5))

display(datos[0, 0])

#### Cómputo paralelo con `map`

La función *built-in* `map` de Python aplica una función sobre una secuencia de datos uno por uno

En general, si vemos un `map` en nuestro código, paralelizarlo es muy sencillo

*ipyparallel* provee una versión paralela de [`map`](https://ipyparallel.readthedocs.io/en/latest/api/ipyparallel.html#ipyparallel.DirectView.map) que se ocupa sobre una vista

    rc[:].map(f, *sequences, block=self.block)
    


In [ ]:
# Map de Python
resultado_serial = list(map(lambda x: x, range(32)))

# Map de ipyparallel
resultado_paralelo = dview.map(lambda x: x, range(32))

# Resultados
np.allclose(resultado_serial, resultado_paralelo)

Podemos entregar iteradores para más de un argumento

Los iteradores deben ser del mismo largo (de lo contrario la secuencia más corta manda)

In [ ]:
dview.map(lambda x, y, z: x + y + z, range(10), range(10), range(10))

Si tenemos una función con algunos argumentos escalares podemos usar partial

In [ ]:
def function_args(x, y, gamma=1):
    import numpy as np
    return np.exp(-gamma*(x-y)**2)

dview.map(partial(function_args, gamma=2), np.random.randn(10), np.random.randn(10))

#### Funciones remotas con decoradores

Podemos crear una función que es siempre ejecutada por los engines usando el decorador `remote`

Por ejemplo:

In [ ]:
@dview.remote(block=True)
def funcion():
    import os
    return os.getpid()

funcion()

#### Funciones paralelas con decoradores

Si tenemos una función que trabaja sobre un arreglo de forma *element-wise* podemos usar el decorador `parallel` para distribuir su carga a los engines

Por ejemplo

In [ ]:
@dview.parallel(block = True)
def funcion(x):
    return x

# Los datos se particionan en 4 grupos (uno por engine)
# Los grupos no son todos del mismo tamaño
funcion(range(10))

También se pueden usar arreglos de numpy

In [ ]:
A = np.random.random((10000, 1000))

@dview.parallel(block=True)
def pmul(A,B):
    return A*B

C_local = A*A

C_remote = pmul(A,A)

(C_local == C_remote).all()


#### Resultado asíncrono

El resultado asíncrono es un objeto de clase [`AsyncResult`](https://ipyparallel.readthedocs.io/en/latest/asyncresult.html#parallel-asyncresult)

Luego podemos usar su función
- `ready` : Retorna un booleano con el estado de la tarea
- `get` : Retorna el resultado

In [ ]:
res_async = dview.map_async(lambda x: x, range(10))
# Resultado asincrono
display(res_async)
# Está listo?
display(res_async.ready())
# Esperamos hasta que este listo y lo recuperamos
while not res_async.ready():
    res = res_async.get()
display(res)

### Paralelizando el fractal de Julia

Propuesto

#### Tópicos extra

- [Magias de ipyparallel](https://ipyparallel.readthedocs.io/en/latest/magics.html)
- Balance de carga automático con la [interfaz Task](https://ipyparallel.readthedocs.io/en/latest/task.html#)
- [Depedencias entre procesos paralelos](https://ipyparallel.readthedocs.io/en/latest/dag_dependencies.html)
- Es posible conectar controladores y engines en distintas máquinas para hacer **computación distribuida** en base a [MPI](https://ipyparallel.readthedocs.io/en/latest/mpi.html)  usando [`ipengine` e `ipcontroller`](https://ipyparallel.readthedocs.io/en/latest/process.html#using-the-ipcontroller-and-ipengine-commands)

# Computación multi-proceso con [joblib](https://joblib.readthedocs.io/en/latest/parallel.html)

Tema para otra iteración

In [ ]:
from joblib import Parallel, delayed
N = 10000000
x = np.random.randn(N)
y = np.random.randn(N)

Parallel(n_jobs=4)(delayed(lambda x, y: np.exp(-1*(x-y)**2))(x, y) for i in range(4))

# Computación multi-hilo con Cython y OpenMP

[OpenMP](https://www.openmp.org/) es una API multiplataforma para computación paralela en C, C++ y Fortran

Ejemplo: En C/C++ se puede escribir un `parallel for` usando directivas de compilador (pragma) de OpenMP 

    #pragma omp parallel for
    for (i = 0; i < N; i++)
        a[i] = 2 * i;

Cython tiene un modulo llamado [`parallel`](http://docs.cython.org/en/latest/src/userguide/parallelism.html) que usa OpenMP como backend

Para ocupar OpenMP desde Cython es necesario 
- instalar OpenMP en el sistema
- compilar el código Cython con `--compile-args=-fopenmp --link-args=-fopenmp`

El modulo provee tres funciones principales

- prange([start,] stop[, step][, nogil=False][, schedule=None[, chunksize=None]][, num_threads=None]): Para escribir un `parallel for`
- parallel(num_threads=None): Para crear un contexto de cómputo paralelo
- threadid(): Para obtener la id del hilo

También se pueden usar funciones de OpenMP importando

    cimport openmp

> El requisito es que las funciones paralelas deben liberar el GIL

En Cython podemos liberar el GIL en una sección de código o en una función con el `keyword` llamado [`nogil`](http://docs.cython.org/en/latest/src/userguide/external_C_code.html#nogil)

### Ejemplo: Cálculo paralelo del kernel Gaussiano entre dos vectores

El kernel Gaussiano se define como 

$$e^{-\gamma (x-y)^2}$$

Asumiremos $\gamma=1$

Escribamos un código en Cython de referencia y otro paralelo con OpenMP para calcular esta función

Primero cargamos la extensión Cython

In [ ]:
%load_ext cython

El código Cython de referencia es similar a lo que vimos la clase anterior

In [ ]:
%%cython -f -c=-O3 -c=-march=native
cimport cython

cdef extern from "math.h":
    double exp (double)
    
@cython.boundscheck(False)
@cython.wraparound(False)
def suma_vectores_cython(double [::1] x, double [::1] y, double [::1] z):
    cdef:
        Py_ssize_t i
        int N = x.shape[0]
    for i in range(N):
        z[i] = exp(-(x[i] - y[i])**2)

En el código paralelo hacemos tres cambios

- Modificamos la magia `%%cython` para compilar contra openmp 
- Agregamos `nogil` en las secciones paralelas
    - Todas las funciones llamadas en la sección paralela deben liberar el GIL 
- Importamos `cython.parallel.prange` para reemplazar el `range` original

Especificamos 4 hilos en `prange`

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force 
# Compilamos con directivas OpenMP
cimport cython

from cython.parallel import prange # Importamos prange

cdef extern from "math.h" nogil: # Liberamos el GIL
    double exp (double)
        
@cython.boundscheck(False)
@cython.wraparound(False)
def suma_vectores_openmp(double [::1] x, double [::1] y, double [::1] z):
    cdef:
        Py_ssize_t i
        int N = x.shape[0]
    with nogil: # Liberamos el GIL
        for i in prange(N, num_threads=4): # For paralelo con 4 hilos
            z[i] = exp(-(x[i] - y[i])**2)

Creamos datos artificiales y medimos los tiempos

In [ ]:
# Creamos un vector grande
N = 1000000
x = np.random.randn(N)
y = np.random.randn(N)
z = np.empty_like(x)
# Tiempos de cómputo
%timeit -r10 -n3 z = np.exp(-(x-y)**2)
%timeit -r10 -n3 suma_vectores_cython(x, y, z)
%timeit -r10 -n3 suma_vectores_openmp(x, y, z)

El resultado es idéntico al de NumPy

In [ ]:
z = np.empty_like(x)
suma_vectores_openmp(x, y, z)
np.allclose(np.exp(-(x-y)**2), z)

### Ejemplo: Fractal de Julia en paralelo con Cython y OpenMP

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force 
import cython
cimport numpy as npc
import numpy as np

from cython.parallel import prange # Importamos prange

ctypedef npc.float32_t TIPOF_t
ctypedef npc.int64_t TIPOI_t

cdef TIPOI_t evaluate_z(TIPOF_t zi, TIPOF_t zr, int maxiters=50, TIPOF_t cr=-0.835, TIPOF_t ci=-0.2321) nogil:
    cdef:
        TIPOI_t nit = 0
        TIPOF_t zi2 = zi**2
        TIPOF_t zr2 = zr**2
        
    while zi2 + zr2 <= 4. and nit < maxiters:
        zi = 2.*zr*zi + ci
        zr = zr2 - zi2 + cr
        zr2 = zr**2
        zi2 = zi**2 
        nit +=1
    return nit

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def make_fractal_cython(int N, TIPOI_t [:, ::1] image_view, int maxiters=50):
    cdef:
        Py_ssize_t i, j
    for i in range(N):
        for j in range(2*N):
            image_view[i, j] = evaluate_z(-1.+i*2./N, -2.+j*2./N, maxiters)
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def make_fractal_openmp(int N, TIPOI_t [:, ::1] image_view, int maxiters=50):
    cdef:
        Py_ssize_t i, j
    
    with nogil:
        for i in prange(N, num_threads=4):        
            for j in range(2*N):
                image_view[i, j] = evaluate_z(-1.+i*2./N, -2.+j*2./N, maxiters)

In [ ]:
N = 1000
image_cython = np.empty(shape=(N, 2*N), dtype=np.int64)
image_openmp = np.empty(shape=(N, 2*N), dtype=np.int64)
%timeit -r3 -n1 make_fractal_cython(N, image_cython)
%timeit -r3 -n1 make_fractal_openmp(N, image_openmp)
np.allclose(image_cython, image_openmp)

# Computación multi-hilo con Cython y Joblib

Para otra iteración

# [PyCUDA](https://documen.tician.de/pycuda/)

Tema para otra iteración

PyCUDA provee acceso a la API de computación paralela de para GPUs de NVIDIA: CUDA

In [ ]:
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from pycuda.elementwise import ElementwiseKernel

import os
if not "/opt/cuda/bin" in os.environ["PATH"]:
    os.environ["PATH"] += os.pathsep + "/opt/cuda/bin"
!nvcc --version

julia_kernel = ElementwiseKernel(
    "int N, int *output, int maxiter",
    """
    {
        float cr = -0.835f;
        float ci = -0.2321f;        
        float zr2, zi2;
        int row = i/(2*N);
        int col = i%(2*N);
        float zr = -2.+col*2./N;
        float zi = -1.+row*2./N;
        output[i] = 0;
        for(int nit = 0; nit < maxiter; nit++) {
            zr2 = zr*zr;
            zi2 = zi*zi;
            if (zr2 + zi2 > 4.0f){
                break;
            }
            zi = 2*zr*zi + ci;
            zr = zr2 - zi2 + cr;
            output[i] += 1;
        }
        
    }
    """, "julia")


N = 10000
image_gpu = gpuarray.to_gpu(np.empty(shape=(2*N*N,), dtype=np.int32))
%timeit -r10 -n3 julia_kernel(N, image_gpu, 50); image_gpu.get()

In [ ]:
N = 20000
image_gpu = gpuarray.to_gpu(np.empty(shape=(2*N*N,), dtype=np.int32))
julia_kernel(N, image_gpu, 50)
image_cpu = image_gpu.get()

fig, ax = plt.subplots(figsize=(8, 4))
ax.axis('off')
ax.imshow(image_cpu.reshape(N, 2*N));

# Magister en Informática: Computación de alto rendimiento

Ciertos problemas son tan extensos que para solucionarlos en un tiempo razonable se requiere poder de cómputo y/o memoria superior a la ofrecida por un computador de arquitectura tradicional

Si el problema es separable entonces puede resolverse de forma eficiente usando computación paralela y/o distribuida

La computación de alto rendimiento o *high-performance computing* (HPC) es la disciplina que se dedica a diseñar algoritmos eficientes que utilizan arquitecturas paralelas/distribuidas avanzadas

En HPC también se investiga la utilización eficiente de co-procesadores de alto paralelismo como son los procesadores de tipo *many-core* (Xeon-Phi) y las Graphical Processing Units (GPU, Nvidia, AMD)

El Magíster en Informática de la UACh tiene a HPC como una de sus áreas principales

Más información en: http://www.ingenieria.uach.cl/index.php/postgrado/magister-en-informatica

http://acme.byu.edu/wp-content/uploads/2015/10/Parallel1.pdf